In [1]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

In [2]:
API_URL = 'http://10.92.52.255:5000/'

stuID = 28374

In [3]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

In [4]:
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcd = b
    return gcd, x, y

def modinv(a, m):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # modular inverse does not exist
    else:
        return x % m

def Setup():
    E = Curve.get_curve('secp256k1')
    return E

def KeyGen(E):
    n = E.order
    P = E.generator
    sA = randint(1,n-1)
    QA = sA*P
    return sA, QA

def SignGen(message, E, sA):
    n = E.order
    P = E.generator
    k = randint(1, n-2)
    R = k*P
    r = R.x % n
    h = int.from_bytes(SHA3_256.new(r.to_bytes((r.bit_length()+7)//8, byteorder='big')+message).digest(), byteorder='big')%n
    s = (sA*h + k) % n
    return h, s

def SignVer(message, h, s, E, QA):
    n = E.order
    P = E.generator
    V = s*P - h*QA
    v = V.x % n
    h_ = int.from_bytes(SHA3_256.new(v.to_bytes((v.bit_length()+7)//8, byteorder='big')+message).digest(), byteorder='big')%n
    if h_ == h:
        return True
    else:
        return False


#server's Identitiy public key
IKey_Ser = Point(93223115898197558905062012489877327981787036929201444813217704012422483432813 , 8985629203225767185464920094198364255740987346743912071843303975587695337619, E)

def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())
    print(response.json())

def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    print(response.json())

def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    print(response.json())

############## The new functions of phase 2 ###############

#Pseudo-client will send you 5 messages to your inbox via server when you call this function
def PseudoSendMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "PseudoSendMsg"), json = mes)		
    print(response.json())

#Get your messages. server will send 1 message from your inbox
def ReqMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqMsg"), json = mes)	
    print(response.json())	
    if((response.ok) == True): 
        res = response.json()
        return res["IDB"], res["OTKID"], res["MSGID"], res["MSG"], res["EK.X"], res["EK.Y"]

#Get the list of the deleted messages' ids.
def ReqDelMsg(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.get('{}/{}'.format(API_URL, "ReqDelMsgs"), json = mes)      
    print(response.json())      
    if((response.ok) == True): 
        res = response.json()
        return res["MSGID"]

#If you decrypted the message, send back the plaintext for checking
def Checker(stuID, stuIDB, msgID, decmsg):
    mes = {'IDA':stuID, 'IDB':stuIDB, 'MSGID': msgID, 'DECMSG': decmsg}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "Checker"), json = mes)		
    print(response.json())


In [17]:
def GenerateKey(pri=-1):
  if pri == -1:
    pri = random.randint(0,n-1)
  pub = pri * P
  return pri, pub

In [18]:
IKey_Pr = int.from_bytes(b"this is a very very very secret key!", byteorder="big") % n
IKey_Pub = IKey_Pr * P

In [19]:
IKey_Pr

14660324670243075630000635893444662718789579794562255004660387285071415640046

In [20]:
stuID_signed = SignGen(stuID.to_bytes(math.ceil(stuID.bit_length()/8), "big"), E, IKey_Pr)
SignVer(stuID.to_bytes(math.ceil(stuID.bit_length()/8),"big"), stuID_signed[0], stuID_signed[1], E, IKey_Pub)

True

In [21]:
IKRegReq(stuID_signed[0], stuID_signed[1], IKey_Pub.x,  IKey_Pub.y)

Sending message is:  {'ID': 28374, 'H': 14003361747863711575107485847218966247208491174002245767442180511820528443999, 'S': 26129671104882079018878570807775261477349243389097282567295507801782727186724, 'IKPUB.X': 59789841742109204850564918020815376627555014601137729807885099765009802042661, 'IKPUB.Y': 57547776729331717451156309143069826691328908716635831074670254759332423229699}
We have sent a verification code to your SU email. If you cant see the email, check your spam folder


In [22]:
CODE = 565496

In [23]:
IKRegVerify(CODE)

Sending message is:  {'ID': 28374, 'CODE': 565496}
Registered successfully


In [24]:
RCODE = 887816
#ResetIK(RCODE)

In [25]:
SPK_A_Pri, SPK_A_Pub = GenerateKey(11786423727943086685271019354073903964054819384868285139085017019872442465785)
print(SPK_A_Pri)

11786423727943086685271019354073903964054819384868285139085017019872442465785


In [26]:
SPK_A_message = SPK_A_Pub.x.to_bytes(math.ceil(SPK_A_Pub.x.bit_length()/8), 'big') + SPK_A_Pub.y.to_bytes(math.ceil(SPK_A_Pub.y.bit_length()/8), 'big')
SPK_A_h, SPK_A_s = SignGen(SPK_A_message, E, IKey_Pr)
SPK_S_X, SPK_S_Y, SPK_S_h, SPK_S_s = SPKReg(SPK_A_h, SPK_A_s, SPK_A_Pub.x, SPK_A_Pub.y)
SPK_S_Pub = Point(SPK_S_X, SPK_S_Y, E)

Sending message is:  {'ID': 28374, 'H': 50139487143911887391897777232166234117231778680325192797770587997492209967547, 'S': 85185251379668123779860426787070865792652845223706050714320739072194942391518, 'SPKPUB.X': 31229216498439133952347392112572115637714405472383255423741905893245939825627, 'SPKPUB.Y': 15829632064927002234237681052859941715925515902168887028034362059443643752689}


In [27]:
# Generate HMAX Key
T = SPK_A_Pri * SPK_S_Pub
U = b'CuriosityIsTheHMACKeyToCreativity' + T.y.to_bytes(math.ceil(T.y.bit_length()/8), 'big') + T.x.to_bytes(math.ceil(T.x.bit_length()/8), 'big')
K_HMAC = SHA3_256.new(U).digest()

In [28]:
# Generate OTKs
ResetOTK(stuID_signed[0], stuID_signed[1])
OTKs=[]
for i in range(10):
  priv, pub = GenerateKey()
  m = pub.x.to_bytes(math.ceil(pub.x.bit_length()/8), 'big') + pub.y.to_bytes(math.ceil(pub.y.bit_length()/8), 'big')
  hmaci = HMAC.new(K_HMAC, m, digestmod=SHA256).hexdigest()
  OTKReg(i, pub.x, pub.y, hmaci)
  OTKs.append(priv)

Sending message is:  {'ID': 28374, 'H': 14003361747863711575107485847218966247208491174002245767442180511820528443999, 'S': 26129671104882079018878570807775261477349243389097282567295507801782727186724}
All OTKs deleted !
Sending message is:  {'ID': 28374, 'KEYID': 0, 'OTKI.X': 101457935187289959549774382001967187134295054504843516554480847645073999987645, 'OTKI.Y': 11911560202403333263968076013219746668260855346719311704286896541245349581874, 'HMACI': '5c00bff522d437ba52a9c1530d45085d1b48fbba21fb0edaafaae2c5964755f7'}
OTK with ID number 0 is registered successfully
Sending message is:  {'ID': 28374, 'KEYID': 1, 'OTKI.X': 3426649075372119385279104271382046654625026328089963360978174516973254203842, 'OTKI.Y': 97822467676486121885870613333710815293671317322411685955530951191424376194970, 'HMACI': '575d718ad531b8a61294d1e4a514f5f190956dfcdad3675a02cdae9587d9183a'}
OTK with ID number 1 is registered successfully
Sending message is:  {'ID': 28374, 'KEYID': 2, 'OTKI.X': 482132558519239060581

In [29]:
PseudoSendMsg(stuID_signed[0], stuID_signed[1])

Sending message is:  {'ID': 28374, 'H': 14003361747863711575107485847218966247208491174002245767442180511820528443999, 'S': 26129671104882079018878570807775261477349243389097282567295507801782727186724}
Your favourite pseudo-client sent you 5 messages. You can get them from the server


In [30]:
def GetSessionKey(OTK, EK):
  T = OTK * EK
  U = T.x.to_bytes(math.ceil(T.x.bit_length()/8), "big") + T.y.to_bytes(math.ceil(T.y.bit_length()/8), "big") + b'ToBeOrNotToBe'
  KS = SHA3_256.new(U).digest()
  return KS

In [31]:
def GetKDF(K_KDF):
  K_ENC = SHA3_256.new(K_KDF + b'YouTalkingToMe').digest()
  K_HMAC = SHA3_256.new(K_KDF + K_ENC + b'YouCannotHandleTheTruth').digest()
  K_KDF_N = SHA3_256.new(K_ENC + K_HMAC + b'MayTheForceBeWithYou').digest()
  return K_ENC, K_HMAC, K_KDF_N

In [32]:
def VerifyHMAC(m, K_HMAC):
  nonce = m[:8]
  hmac = m[-32:]
  msg = m[8:-32]

  hmac_new = HMAC.new(K_HMAC, msg,digestmod=SHA256)
  hmac_final = hmac_new.digest()

  if(hmac == hmac_final):
    return m[:-32]
  return "INVALIDHMAC"

In [33]:
def AesDecrypt(ctext, key):
  cipher = AES.new(key, AES.MODE_CTR, nonce=ctext[:8])
  ptext = cipher.decrypt(ctext[8:])
  ptext = ptext.decode('UTF-8')
  print("plaintext:", ptext)
  return ptext

In [37]:
for i in range(5):
  stuIDB, otkID, msgID, msg, EK_X, EK_Y = ReqMsg(stuID_signed[0], stuID_signed[1])
  KS = GetSessionKey(OTKs[otkID], Point(EK_X, EK_Y, E))
  if i == 0:
    K_ENC, K_HMAC, K_KDF_N = GetKDF(KS)
  else:
    K_ENC, K_HMAC, K_KDF_N = GetKDF(K_KDF_N)

  msg = VerifyHMAC(msg.to_bytes(math.ceil(msg.bit_length()/8),"big"), K_HMAC)
  
  if(msg!="INVALIDHMAC"):
    plaintext = AesDecrypt(msg, K_ENC)
    Checker(stuID, stuIDB, msgID, plaintext)
  else:
    print("Invalid")
    Checker(stuID, stuIDB, msgID, "INVALIDHMAC")

Sending message is:  {'ID': 28374, 'H': 14003361747863711575107485847218966247208491174002245767442180511820528443999, 'S': 26129671104882079018878570807775261477349243389097282567295507801782727186724}
There is no message in your message box. Pseudo client will send you messages if you call this method: PseudoSendMsg


TypeError: cannot unpack non-iterable NoneType object

In [36]:
ReqDelMsg(stuID_signed[0],stuID_signed[1])

Sending message is:  {'ID': 28374, 'H': 14003361747863711575107485847218966247208491174002245767442180511820528443999, 'S': 26129671104882079018878570807775261477349243389097282567295507801782727186724}
There are no deleted messages in the last message block !!
